In [25]:
import os
from typing import List
from fastapi import FastAPI
from pydantic import BaseModel
from datetime import datetime
from sqlalchemy import create_engine
import pandas as pd
from catboost import CatBoostClassifier
from sqlalchemy import create_engine
import numpy as np

In [26]:
def batch_load_sql(query: str) -> pd.DataFrame:
    CHUNKSIZE = 200000
    engine = create_engine(
        "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
    )
    conn = engine.connect().execution_options(stream_results=True)
    chunks = []
    for chunk_dataframe in pd.read_sql(query, conn, chunksize=CHUNKSIZE):
        chunks.append(chunk_dataframe)
    conn.close()
    return pd.concat(chunks, ignore_index=True)

def load_features():
    return batch_load_sql("""SELECT * FROM "shatalov_dmitry_user" """)
def load_text():
    return batch_load_sql("""SELECT * FROM "shatalov_dmitry_text" """)

In [27]:
user_data = load_features()
posts_data = load_text()

In [28]:
user_data = user_data.drop("index", axis=1)

In [29]:
posts_data = posts_data.drop("index", axis=1)

In [30]:
# date_string = "2023-02-10 15:26:40"
# datetime_object = datetime.strptime(date_string, "%Y-%m-%d %H:%M:%S")
# print(datetime_object)

2023-02-10 15:26:40


In [13]:
# int(datetime_object.timestamp())

1676028400

In [7]:
# user_data[user_data['user_id'] == 202]['timestamp'] = datetime_object.timestamp()

C:\Users\Диман\AppData\Local\Temp\ipykernel_932\3705897018.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_data[user_data['user_id'] == 202]['timestamp'] = datetime_object.timestamp()


In [31]:
# user_data.loc[user_data['user_id'] == 202, "timestamp"] = int(datetime_object.timestamp())


In [7]:
# user_data = user_data.reset_index()
# posts_data = posts_data

In [32]:
#data_for_prediction = pd.concat([user_data[user_data['user_id'] == 202].reset_index(), posts_data.iloc[[0]].drop(['text'], axis=1)], axis=1).drop(["index"], axis=1)
# data_for_prediction = pd.concat([user_data[user_data['user_id'] == 202].reset_index(), posts_data.drop(['text'], axis=1)], axis=1).drop(["index"], axis=1)

In [33]:
# data_for_prediction.head()

,user_id,gender,age,country,exp_group,os,source,timestamp,post_id,topic,...,90,91,92,93,94,95,96,97,98,99
0,202.0,1.0,17.0,Russia,4.0,Android,ads,1.676028e+09,1,business,...,-0.027707,0.009552,-0.008999,-0.013628,-0.006564,-0.010809,-0.033995,-0.037561,-0.001322,-0.013360
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,business,...,0.026562,0.026715,0.022422,0.011496,-0.030049,-0.013404,-0.006376,0.006521,0.013236,-0.014821
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,business,...,0.003895,-0.026901,-0.059054,0.028854,-0.045230,0.018362,-0.000664,0.047550,-0.038122,0.002763
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,business,...,-0.029090,-0.008279,-0.047934,-0.010357,-0.006429,-0.010600,-0.018037,-0.042471,-0.019968,0.031709
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,business,...,-0.009430,0.012673,-0.017110,-0.011624,-0.009426,-0.003026,0.019623,-0.026298,0.011045,0.006989


In [41]:
# data_for_prediction = data_for_prediction.fillna(method="ffill")

In [45]:
# data_for_prediction.insert(9, "hour", datetime_object.hour)

In [42]:
# data_for_prediction

,user_id,gender,age,country,exp_group,os,source,timestamp,post_id,topic,...,90,91,92,93,94,95,96,97,98,99
0,202.0,1.0,17.0,Russia,4.0,Android,ads,1.676028e+09,1,business,...,-0.027707,0.009552,-0.008999,-0.013628,-0.006564,-0.010809,-0.033995,-0.037561,-0.001322,-0.013360
1,202.0,1.0,17.0,Russia,4.0,Android,ads,1.676028e+09,2,business,...,0.026562,0.026715,0.022422,0.011496,-0.030049,-0.013404,-0.006376,0.006521,0.013236,-0.014821
2,202.0,1.0,17.0,Russia,4.0,Android,ads,1.676028e+09,3,business,...,0.003895,-0.026901,-0.059054,0.028854,-0.045230,0.018362,-0.000664,0.047550,-0.038122,0.002763
3,202.0,1.0,17.0,Russia,4.0,Android,ads,1.676028e+09,4,business,...,-0.029090,-0.008279,-0.047934,-0.010357,-0.006429,-0.010600,-0.018037,-0.042471,-0.019968,0.031709
4,202.0,1.0,17.0,Russia,4.0,Android,ads,1.676028e+09,5,business,...,-0.009430,0.012673,-0.017110,-0.011624,-0.009426,-0.003026,0.019623,-0.026298,0.011045,0.006989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,202.0,1.0,17.0,Russia,4.0,Android,ads,1.676028e+09,7315,movie,...,-0.012011,-0.049446,-0.035269,-0.000707,-0.000838,-0.041760,0.010854,-0.029259,0.054002,-0.026419
7019,202.0,1.0,17.0,Russia,4.0,Android,ads,1.676028e+09,7316,movie,...,-0.003053,0.007155,0.041740,0.046052,0.027412,0.055240,0.011004,-0.017853,-0.035655,0.025860
7020,202.0,1.0,17.0,Russia,4.0,Android,ads,1.676028e+09,7317,movie,...,-0.001801,-0.021358,-0.028719,0.039713,-0.004553,0.027980,0.035541,-0.013655,0.037083,-0.003795
7021,202.0,1.0,17.0,Russia,4.0,Android,ads,1.676028e+09,7318,movie,...,-0.009251,-0.034486,-0.013391,-0.023257,0.003686,-0.004353,-0.030709,-0.017063,-0.028726,0.016211


In [126]:

# date_string = "2023-02-10 15:26:40"
# datetime_object = datetime.strptime(date_string, "%Y-%m-%d %H:%M:%S")
# print(datetime_object)

2023-02-10 15:26:40


In [ ]:
# data_for_prediction['hour'] = user_behavior_copy.datetime_object.dt.hour

In [127]:
# type(datetime_object)

datetime.datetime

In [43]:
from_file = CatBoostClassifier()  # здесь не указываем параметры, которые были при обучении, в дампе модели все есть

from_file.load_model("catboost_model_3")

In [15]:
# data_for_prediction.head()

,user_id,gender,age,country,exp_group,os,source,post_id,topic,hour,...,90,91,92,93,94,95,96,97,98,99
0,202,1,17,Russia,4,Android,ads,1,business,15,...,-0.027707,0.009552,-0.008999,-0.013628,-0.006564,-0.010809,-0.033995,-0.037561,-0.001322,-0.01336


In [70]:
# pred.shape

(7023, 2)

In [75]:
# data_for_prediction[["post_id"]].shape

(7023, 1)

In [ ]:
class PostGet(BaseModel):
    id: int
    text: str
    topic: str

    class Config:
        orm_mode = True

In [ ]:
app = FastAPI()
@app.get("/post/recommendations/", response_model=List[PostGet])
def recommended_posts(
		id: int,
		time: datetime,
		limit: int = 5) -> List[PostGet]:
    user_data.loc[user_data['user_id'] == id, "timestamp"] = int(time.timestamp())
    data_for_prediction = pd.concat([user_data[user_data['user_id'] == id].reset_index(), posts_data.drop(['text'], axis=1)], axis=1).drop(["index"], axis=1)
    data_for_prediction = data_for_prediction.fillna(method="ffill")
    data_for_prediction.insert(9, "hour", time.hour)
    pred = from_file.predict_proba(data_for_prediction)
    pred = np.append(arr=pred, values = data_for_prediction[["post_id"]],axis=1)
    pred = pd.DataFrame(pred)
    pred = pred.sort_values(by=1, ascending=False)
    answers = []
    for i in range(limit):
        post_id = int(pred.iloc[i][2])
        answers.append({"id":post_id , "text": posts_data.loc[posts_data["post_id"] == post_id].text.iloc[0],
        "topic" : posts_data.loc[posts_data["post_id"] == post_id].topic.iloc[0]})

In [118]:
# def recom(id, time, limit=5):
#     dict_of_probabilities = {}
#     list_of_probabilities = []
#     answers = []
#     for post in range(100):
#         user_data.loc[user_data['user_id'] == id, "timestamp"] = int(time.timestamp())
#         data_for_prediction = pd.concat([user_data[user_data['user_id'] == id].reset_index(), posts_data.iloc[[post]].reset_index().drop(['text'], axis=1)], axis=1).drop(["index"], axis=1)
#         data_for_prediction.insert(9, "hour", datetime_object.hour)
#
#          #сохраним в словарь значение вероятности для класса 1 как ключ и  как значение пост айди
#         dict_of_probabilities[float(round((from_file.predict_proba(data_for_prediction)[0][1]), 5))] = posts_data.iloc[post]["post_id"]
#
#
#
#     for key in dict_of_probabilities.keys():
#         list_of_probabilities.append(key)
#
#     for i in range(limit):
#         print(dict_of_probabilities[list_of_probabilities[i]])
#         print(a)
#         answers.append({"id":dict_of_probabilities[list_of_probabilities[i]]})
#         # , "text": posts_data.text[posts_data["post_id"] == dict_of_probabilities[list_of_probabilities[i]]][0],
#         #                "topic" : posts_data.topic[posts_data["post_id"] == dict_of_probabilities[list_of_probabilities[i]]][0]})
#     return answers